In [1]:
import sqlite3

def create_connection(db_file):
    """ Create a database connection to the SQLite database specified by db_file """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
    return None

def create_table(conn):
    """ Create a STUDENTS table if it doesn't exist in the passed database connection """
    try:
        sql_create_students_table = """
        CREATE TABLE IF NOT EXISTS STUDENTS (
            Id INTEGER PRIMARY KEY,
            Name TEXT NOT NULL,
            Age INTEGER NOT NULL
        );
        """
        cursor = conn.cursor()
        cursor.execute(sql_create_students_table)
    except sqlite3.Error as e:
        print(e)

def insert_or_update(conn, id, name, age):
    """ Insert a new student or update an existing one based on whether the ID already exists """
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM STUDENTS WHERE Id=?", (id,))
        is_record_exist = cursor.fetchone() is not None # check row or data
        if is_record_exist:
            cursor.execute("UPDATE STUDENTS SET Name=?, Age=? WHERE Id=?", (name, age, id))
        else:
            cursor.execute("INSERT INTO STUDENTS (Id, Name, Age) VALUES (?, ?, ?)", (id, name, age))
        conn.commit() # save (commit) the transaction
    except sqlite3.Error as e:
        print(e)

def main():
    database = "sqlite.db"
    conn = create_connection(database)
    if conn:
        create_table(conn)  # create STUDENTS table
        try:
            id = int(input('Enter user ID: '))
            name = input('Enter User Name: ')
            age = int(input('Enter User Age: '))
            insert_or_update(conn, id, name, age)
        finally:
            conn.close()  # close the connection
    else:
        print("Error! Cannot create the database connection.")

if __name__ == '__main__':
    main()


ValueError: invalid literal for int() with base 10: ''

In [ ]:
import cv2
import os
import numpy as np

# Path where the captured images will be saved
dataset_path = 'dataset/'
if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)

# Face detector setup
face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
  
# it detect and crop while capturing image from camera
# Initialize the webcam
cam = cv2.VideoCapture(0)
cam.set(3, 640)  # Set video width
cam.set(4, 480)  # Set video height

# Enter the ID for the person we're capturing data for
face_id = input('\n Enter user ID:  ')

print("\n [INFO] Initializing face capture. Look at the camera and wait ...")
count = 0

# Start capturing faces
while True:
    ret, img = cam.read()
    if not ret:
        break
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.3, 5)

    # Draw rectangles around faces
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)     
        count += 1

        # Save the captured image into the datasets folder
        cv2.imwrite(f"{dataset_path}User.{face_id}.{count}.jpg", gray[y:y+h, x:x+w])

        cv2.imshow('image', img)

    k = cv2.waitKey(100) & 0xff  # Press 'ESC' to exit
    if k == 27:
        break
    elif count >= 60:
        break

# Cleanup
print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()

# Now, training the model from the captured images
from cv2 import face
recognizer = face.LBPHFaceRecognizer_create()

def get_images_and_labels(path):
    image_paths = [os.path.join(path, f) for f in os.listdir(path)]
    face_samples = []
    ids = []

    for image_path in image_paths:
        PIL_img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        img_numpy = np.array(PIL_img, 'uint8')
        id = int(os.path.split(image_path)[-1].split(".")[1])
        faces = face_detector.detectMultiScale(img_numpy)
        for (x, y, w, h) in faces:
            face_samples.append(img_numpy[y:y+h, x:x+w])
            ids.append(id)
    return face_samples, ids

# =============================================================================================
# =============================================================================================

print("\n [INFO] Training faces. It will take a few seconds. Wait ...")
faces, ids = get_images_and_labels(dataset_path)
recognizer.train(faces, np.array(ids))

# Save the model into a file
recognizer.save('trainer.yml')
print("\n [INFO] {len(np.unique(ids))} faces trained. Exiting Program.")



In [ ]:
import cv2
import numpy as np
import sqlite3

def getprofile(id):
    """ Retrieve user profile by ID from the database. """

    try:
        conn = sqlite3.connect("sqlite.db")
        cursor = conn.execute("SELECT * FROM STUDENTS WHERE id=?", (id,))
        profile = None
        for row in cursor:
            profile = row
        return profile
    finally:
        conn.close()

facedetect = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
cam = cv2.VideoCapture(0)
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer.yml')

try:
    while True:
        ret, img = cam.read()
        if not ret:
            print("Failed to capture frame")
            break
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = facedetect.detectMultiScale(gray, 1.3, 5) 

        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            id, conf = recognizer.predict(gray[y:y + h, x:x + w])
            profile = getprofile(id)
            
            if profile:
                text = f"ID: {id} - Name: {profile[1]} - Age: {profile[2]}"
                cv2.putText(img, text, (x, y + h + 60), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 255, 255), 2)

        cv2.imshow("FACE", img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    cam.release()
    cv2.destroyAllWindows()
